**Step 1 – Writing Python Program**

In [60]:
#import requred libraries
import random
import datetime
import logging
import time
import psutil
import numpy as np
import pandas as pd
import matplotlib

**Step 1.1 : Customer Generation**

In [61]:
# Define age, marital status, number of children, education level mapping
age_ranges = [(20, 30), (31, 60), (60, 80)]
marital_status_probs = [(0.75, 0.25), (0.25, 0.75), (0.5, 0.5)]
children_probs = [(0.4, 0.3, 0.2, 0.1, 0), (0.1, 0.3, 0.3, 0.2, 0.1)]
education_probs = [(0.1, 0.5, 0.3, 0.1, 0), (0.1, 0.5, 0.3, 0.05, 0.05), (0.1, 0.5, 0.25, 0.1, 0.05)]
cust_ids=[]
# Generate 7-digit customer ID
def generate_customer_id():
    id = random.randint(1000000, 9999999)
    if id not in cust_ids:
        cust_ids.append(id)
        return id
    return generate_customer_id()

# Generate random age
def generate_age():
    return random.randint(21, 80)

# Generate random gender
def generate_gender():
    return random.choice(['Male', 'Female'])

# Generate marital status based on age
def generate_marital_status(age):
    if age <= 30:
        return random.choices(['Single', 'Married'], weights=[0.75, 0.25])[0]
    elif age <= 60:
        return random.choices(['Single', 'Married'], weights=[0.25, 0.75])[0]
    else:
        return random.choices(['Single', 'Married'], weights=[0.5, 0.5])[0]

# Generate number of children based on age
def generate_children(age,marital_status):
    if marital_status=="Married":
        if age <= 40:
            return random.choices(range(5), weights=[0.4, 0.3, 0.2, 0.1, 0])[0]
        else:
            return random.choices(range(5), weights=[0.1, 0.3, 0.3, 0.2, 0.1])[0]
    return 0

# Generate education level based on age
def generate_education(age):
    if age <= 25:
        return random.choices(range(5), weights=[0.1, 0.5, 0.3, 0.1, 0])[0]
    elif age <= 35:
        return random.choices(range(5), weights=[0.1, 0.5, 0.3, 0.05, 0.05])[0]
    else:
        return random.choices(range(5), weights=[0.1, 0.5, 0.25, 0.1, 0.05])[0]

# Generate annual income based on education level and age
def generate_annual_income(education_level, age):
    return 40 * 52 * (15 +int(education_level or 0) * 10 + (age / 10) * 2)

# Generate number of accounts based on marital status and number of children
def generate_num_accounts(marital_status, num_children):
    return  int(num_children or 0) + 1 if marital_status == 'Single' else 1 + int(num_children or 0) + 1

# Generate total credit line based on number of accounts and annual income
def generate_total_credit_line(num_accounts, annual_income):
    return num_accounts * (annual_income / 10)

logging.basicConfig(filename='Logs.log', level=logging.INFO,format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# Start logging
logging.info("Step 1 customer_generation started")
# Start time measurement
start_time = time.time()

# Generate 20,000 customers
customers = []
for _ in range(20000):
    customer_id = generate_customer_id()
    age = generate_age()
    gender = generate_gender()
    marital_status = generate_marital_status(age)
    num_children = generate_children(age,marital_status)
    education_level = generate_education(age)
    annual_income = generate_annual_income(education_level, age)
    num_accounts = generate_num_accounts(marital_status, num_children)
    total_credit_line = generate_total_credit_line(num_accounts, annual_income)

    customers.append({
        'Customer ID': customer_id,
        'Age': age,
        'Gender': gender,
        'Marital Status': marital_status,
        'Number of Children': num_children,
        'Education Level': education_level,
        'Annual Income': annual_income,
        'Number of Accounts': num_accounts,
        'Total Credit Line': total_credit_line
    })
    logging.info(f"Generating customer {_} completed")
# Print the first customer for verification
print(customers[0])

{'Customer ID': 8676767, 'Age': 58, 'Gender': 'Male', 'Marital Status': 'Married', 'Number of Children': 2, 'Education Level': 2, 'Annual Income': 96928.0, 'Number of Accounts': 4, 'Total Credit Line': 38771.2}


**Step 1.2 : Customer Account Generation**

In [62]:
# Define constants
MIN_INTEREST_RATE = 15
MAX_INTEREST_RATE = 30

# Function to generate random date before January 1st, 2022
def generate_random_date(age):
    # Set today's date as January 1st, 2022
    current_date = datetime.datetime(2022, 1, 1)
    # Calculate the earliest possible birth year based on the maximum account age difference
    min_birth_year = current_date.year - age - 20
    # Calculate the latest possible birth year based on the customer's current age
    max_birth_year = current_date.year - age
    # Define the earliest and latest possible dates for account opening
    earliest_date = datetime.datetime(min_birth_year + 20, 1, 1)  # January 1st after turning 20
    latest_date = datetime.datetime(max_birth_year, 12, 31)  # December 31st of the birth year
    # If the earliest date is after the latest date, return None indicating impossibility
    if earliest_date > latest_date:
        return None
    # Calculate the range of days between the earliest and latest possible dates
    days_range = (latest_date - earliest_date).days
    # Generate a random number of days between the calculated range
    random_days = random.randint(0, days_range)
    # Calculate the random date within the specified range
    random_date = earliest_date + datetime.timedelta(days=random_days)
    return random_date.strftime("%Y-%m-%d")

# Function to generate account number
def generate_account_number(customer_id, account_index):
    return int(str(customer_id) + str(account_index))

# Function to generate account credit line
def generate_account_credit_line(total_credit_line):
    return random.uniform(0.05, 0.5) * total_credit_line

# Function to generate annual fee
def generate_annual_fee(account_credit_line):
    return account_credit_line * 0.01

# Function to generate annual interest rate
def generate_annual_interest_rate():
    return random.uniform(0.15, 0.3)
# Start logging
logging.info("Step 2 customer_account_generation started")
customer_count=1
# Generate account information for each customer
for customer in customers:
    customer['Accounts'] = []
    logging.info(f"Generating account for Customer {customer_count} started")

    for i in range(customer['Number of Accounts']):
        date_opened = generate_random_date(customer['Age'])
        account_number = generate_account_number(customer['Customer ID'], i + 1)
        account_credit_line = generate_account_credit_line(customer['Total Credit Line'])
        annual_fee = generate_annual_fee(account_credit_line)
        annual_interest_rate = generate_annual_interest_rate()

        account_info = {
            'Date Opened': date_opened,
            'Account Number': account_number,
            'Credit Line': account_credit_line,
            'Annual Fee': annual_fee,
            'Annual Interest Rate': annual_interest_rate
        }
        customer['Accounts'].append(account_info)
        logging.info(f"Generating account for Customer {customer_count} completed")
        customer_count+=1

# Print the account information for the first customer for verification
print(customers[0])

{'Customer ID': 8676767, 'Age': 58, 'Gender': 'Male', 'Marital Status': 'Married', 'Number of Children': 2, 'Education Level': 2, 'Annual Income': 96928.0, 'Number of Accounts': 4, 'Total Credit Line': 38771.2, 'Accounts': [{'Date Opened': '1964-02-04', 'Account Number': 86767671, 'Credit Line': 16998.146822747625, 'Annual Fee': 169.98146822747626, 'Annual Interest Rate': 0.19995419823844512}, {'Date Opened': '1964-06-06', 'Account Number': 86767672, 'Credit Line': 4721.972697272988, 'Annual Fee': 47.21972697272988, 'Annual Interest Rate': 0.17060466899810472}, {'Date Opened': '1964-05-22', 'Account Number': 86767673, 'Credit Line': 13773.971882199008, 'Annual Fee': 137.7397188219901, 'Annual Interest Rate': 0.23446890123557945}, {'Date Opened': '1964-12-17', 'Account Number': 86767674, 'Credit Line': 13751.893141824188, 'Annual Fee': 137.5189314182419, 'Annual Interest Rate': 0.2778489676550822}]}


**Step 1.3 : Customer Account Activity Generation**

In [63]:
def generate_random_days_between_uses():
    return random.randint(0, 7)

def simulate_customer_payment(account, today):
    payment_scenario = random.choice(['exact', 'full_balance', 'proportion', 'payment_period'])
    minimum_amount_due = 0.0  # Initialize minimum_amount_due
    
    if payment_scenario == 'exact':
        
        account["Credit Line"]+=account["CurrentBalance"]
        account['Transactions'].append({
            'Type': 'Payment',
            'PaymentAmount': account['CurrentBalance']
        })
        account['CurrentBalance'] = 0.0
        
    elif payment_scenario == 'full_balance':
        account['CurrentBalance'] -= account['CurrentBalance']
        account["Credit Line"]+=account["CurrentBalance"]
        account['Transactions'].append({
            'Type': 'Payment',
            'PaymentAmount': account['CurrentBalance']
        })
        
    elif payment_scenario == 'proportion':
        p = random.uniform(0, 1)
        pro=p * account['CurrentBalance']
        account['CurrentBalance'] -=pro 
        account["Credit Line"]+=pro
        account['Transactions'].append({
            'Type': 'Payment',
            'PaymentAmount': pro
        })
        
    elif payment_scenario == 'payment_period':
        if random.random() < 0.1:  # 10% pay entire balance within payment period
            account["Credit Line"]+=account["CurrentBalance"]
            account['Transactions'].append({
                'Type': 'Payment',
                'PaymentAmount': account['CurrentBalance']
            })
            account['CurrentBalance'] = 0.0
        
        elif random.random() < 0.15:  # 15% pay minimum amount due within payment period
            day = random.randint(1, 10)
            if (today.day <= day) and (account['CurrentBalance'] > 0):
                minimum_amount_due = 0.1 * account['CurrentBalance']
                account["Credit Line"]+=minimum_amount_due
                account['Transactions'].append({
                    'Type': 'Payment',
                    'PaymentAmount': minimum_amount_due
                })
                # Reset delinquency counter and past due amount upon payment
                account['DelinquencyCounter'] = 0
                account['PastDueAmount'] = 0.0
            else:
                if today.day > 10 and account['PastDueAmount'] == 0:
                    # Set past due amount and add late payment fee
                    account['PastDueAmount'] = minimum_amount_due
                    account['CurrentBalance'] += 30  # Late Payment Fee
                    account['DelinquencyCounter'] += 1
                    account['Transactions'].append({
                        'Type': 'Late Payment Fee',
                        'FeeAmount': 30
                    })

def simulate_account_activity(account):
    months_range = {1:[1,31], 2:[1,28], 3:[1,31], 4:[1,30], 5:[1,31], 6:[1,30], 7:[1,31], 8:[1,31], 9:[1,30], 10:[1,31], 11:[1,30], 12:[1,31]}
    
    start_date = datetime.datetime(2022, 1, 1)
    end_date = datetime.datetime(2022, 12, 31)
    
    current_date = start_date
    
    while current_date <= end_date:
        days_between_uses = generate_random_days_between_uses()
        current_date += datetime.timedelta(days=days_between_uses)
        
        available_credit_line = account['Credit Line'] - account['CurrentBalance']
        
        if 0.1 * account['Credit Line'] <= available_credit_line:
            available_cash = 0.1 * account['Credit Line']
        else:
            available_cash = available_credit_line
        
        if random.random() < 0.95:  # 95% chance of making a purchase
            if available_credit_line > 0:
                purchase_amount = random.uniform(0, available_credit_line)
                account["Credit Line"]-=purchase_amount
                account['CurrentBalance'] += purchase_amount
                account['Transactions'].append({
                        'Type': 'Purchase',
                        'PurchaseAmount': purchase_amount
                    })
        else:  # 5% chance of taking cash advance
            if available_cash > 0:
                cash_advance_amount = random.uniform(0, available_cash)
                account["Credit Line"]-=cash_advance_amount
                account['CurrentBalance'] += cash_advance_amount
                account['Transactions'].append({
                        'Type': 'Cash Advance',
                        'CashAdvanceAmount': cash_advance_amount
                    })
        
        daily_interest_rate = account['Annual Interest Rate'] / 365
        account['CurrentBalance'] *= (1 + daily_interest_rate)
        
        simulate_customer_payment(account, current_date)
        
        if account['DelinquencyCounter'] == 3 or current_date >= end_date:
            account['AccountStatus'] = 'Closed' if account['DelinquencyCounter'] == 3 else 'Active'
            break
        # Close the account if the delinquency counter reaches 3
        if account['DelinquencyCounter'] == 3:
            account['AccountStatus'] = 'Closed'
            break

def perform_monthly_calculations(account, today):
    closing_balance = account['CurrentBalance']
    minimum_amount_due = 0.1 * closing_balance

    total_purchases_month = sum(transaction['PurchaseAmount'] for transaction in account['Transactions'] if transaction['Type'] == 'Purchase')
    total_cash_advances_month = sum(transaction['CashAdvanceAmount'] for transaction in account['Transactions'] if transaction['Type'] == 'Cash Advance')
    total_payments_month = sum(transaction['PaymentAmount'] for transaction in account['Transactions'] if transaction['Type'] == 'Payment')
    total_interests_month = total_interest_charged(account, today)

    account['MonthlyDetails'] = {
        'ClosingBalance': closing_balance,
        'MinimumAmountDue': minimum_amount_due,
        'TotalPurchasesMonth': total_purchases_month,
        'TotalCashAdvancesMonth': total_cash_advances_month,
        'TotalPaymentsMonth': total_payments_month,
        'TotalInterestsChargedMonth': total_interests_month
    }

def total_interest_charged(account, today):
    start_date = datetime.datetime(today.year, today.month, 1)
    end_date = today

    total_interest = 0
    current_balance = account['CurrentBalance']

    while start_date <= end_date:
        daily_interest_rate = account['Annual Interest Rate'] / 365
        current_balance *= (1 + daily_interest_rate)
        total_interest += current_balance - account['CurrentBalance']
        start_date += datetime.timedelta(days=1)

    return total_interest
customer_count=1
for customer in customers:
    logging.info(f"Simulating account for Customer {customer_count} started")
    for account in customer['Accounts']:
        account['CurrentBalance'] = 0.0
        account['DelinquencyCounter'] = 0
        account['PastDueAmount'] = 0.0
        account['Transactions'] = []
        account['AccountStatus'] = 'Active'
        simulate_account_activity(account)
        today = datetime.datetime(2022, 12, 31)  

        perform_monthly_calculations(account, today)

        logging.info(f"Simulating account for Customer {customer_count} completed")
        customer_count+=1

**Sample Output**

In [64]:
customer_df =  pd.DataFrame(customers)
display(customer_df)

,Customer ID,Age,Gender,Marital Status,Number of Children,Education Level,Annual Income,Number of Accounts,Total Credit Line,Accounts
0,8676767,58,Male,Married,2,2,96928.0,4,38771.2,"[{'Date Opened': '1964-02-04', 'Account Number..."
1,4972248,58,Male,Married,3,2,96928.0,5,48464.0,"[{'Date Opened': '1964-04-26', 'Account Number..."
2,4247349,36,Male,Married,0,0,46176.0,2,9235.2,"[{'Date Opened': '1986-01-01', 'Account Number..."
3,8032300,60,Female,Married,1,0,56160.0,3,16848.0,"[{'Date Opened': '1962-07-08', 'Account Number..."
4,5693366,49,Female,Single,0,1,72384.0,1,7238.4,"[{'Date Opened': '1973-07-16', 'Account Number..."
...,...,...,...,...,...,...,...,...,...,...
19995,7025723,64,Female,Single,0,1,78624.0,1,7862.4,"[{'Date Opened': '1958-09-21', 'Account Number..."
19996,8072848,72,Male,Married,2,1,81952.0,4,32780.8,"[{'Date Opened': '1950-11-06', 'Account Number..."
19997,9510347,67,Male,Married,0,1,79872.0,2,15974.4,"[{'Date Opened': '1955-12-19', 'Account Number..."
19998,4857788,44,Male,Married,2,1,70304.0,4,28121.6,"[{'Date Opened': '1978-12-25', 'Account Number..."


**Step 2 – Functional Testing**

In [65]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Convert the data to a pandas DataFrame


**A. Functional Testing for Cohort information**

In [66]:
'''1. Number of customers in your results'''
num_customers = len(customer_df)
print("\nNumber of customers: ",num_customers)


'''2. The minimum and maximum Customer ID in your results'''
min_customer_id = customer_df['Customer ID'].min()
max_customer_id = customer_df['Customer ID'].max()
print(f"\nMinimum Customer id {min_customer_id}   Maximum Customer id {max_customer_id}")


'''3. Number of unique Customer IDs'''
num_unique_customer_ids = customer_df['Customer ID'].nunique()
print("\n Number of Unique customer Ids: ",num_unique_customer_ids)



'''4. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the age of customers.'''
age_stats = customer_df['Age'].describe()
print("\n Stats on Customer Age:")
display(age_stats)
# age_stats.to_csv('age_stats.csv',index=False)


'''5. Frequency table for Gender of the customers (category, number of observations as frequency, 
percent frequency, cumulative frequency, cumulative percent frequency)'''
gender_freq_table = customer_df['Gender'].value_counts().reset_index()
gender_freq_table.columns = ['Gender', 'Frequency']
gender_freq_table['Percent Frequency'] = (gender_freq_table['Frequency'] / len(customer_df)) * 100
gender_freq_table['Cumulative Frequency'] = gender_freq_table['Frequency'].cumsum()
gender_freq_table['Cumulative Percent Frequency'] = (gender_freq_table['Cumulative Frequency'] / gender_freq_table['Frequency'].sum()) * 100
display(gender_freq_table)
# gender_freq_table.to_csv('gender_freq_table.csv', index=False)


'''6. Frequency table for Marital Status of the customers (category, number of observations as 
frequency, percent frequency, cumulative frequency, cumulative percent frequency)'''
marital_status_freq_table = customer_df['Marital Status'].value_counts().reset_index()
marital_status_freq_table.columns = ['Martial Status', 'Frequency']
marital_status_freq_table['Percent Frequency'] = (marital_status_freq_table['Frequency'] / len(customer_df)) * 100
marital_status_freq_table['Cumulative Frequency'] = marital_status_freq_table['Frequency'].cumsum()
marital_status_freq_table['Cumulative Percent Frequency'] = (marital_status_freq_table['Cumulative Frequency'] / marital_status_freq_table['Frequency'].sum()) * 100
print("\n Martial Status frequency Table:")
display(marital_status_freq_table)
# marital_status_freq_table.to_csv('marital_status_freq_table.csv', index=False)


'''7. Provide percent frequency of Marital Status for the following categories separately:
a. For customers with age in [20, 30]
b. For customers with age in (30, 60]
c. For customers with age in (60, 80]'''
result_df = []

age_ranges = {
    '20-30': (20, 30),
    '30-60': (30, 60),
    '60-80': (60, 80)
}

# Filter data by age ranges and calculate percent frequency of Marital Status
for label, age_range in age_ranges.items():
    lower, upper = age_range
    filtered_data = customer_df[(customer_df['Age'] > lower) & (customer_df['Age'] <= upper)]
    
    total_customers = len(filtered_data)
    marital_status_freq = filtered_data['Marital Status'].value_counts(normalize=True) * 100
    
    for status, percent in marital_status_freq.items():
        result_df.append({'Age Range': label, 'Marital Status': status, 'Percent Frequency': percent})
marital_status_freq_by_age=pd.DataFrame(result_df)
print("\n Martial Stats frequency by Age:")
display(marital_status_freq_by_age)
# marital_status_freq_by_age.to_csv('marital_status_freq_by_age.csv', index=False)


'''8. Frequency table for Number of Children of the customers (category, number of observations 
as frequency, percent frequency, cumulative frequency, cumulative percent frequency)'''
children_freq_table = customer_df['Number of Children'].value_counts(dropna=False).reset_index()
children_freq_table.columns = ['Number of Children', 'Frequency']
children_freq_table['Percent Frequency'] = (children_freq_table['Frequency'] / len(customer_df)) * 100
children_freq_table['Cumulative Frequency'] = children_freq_table['Frequency'].cumsum()
children_freq_table['Cumulative Percent Frequency'] = (children_freq_table['Cumulative Frequency'] / len(customer_df)) * 100
print("\n Children frequency Table:")
display(children_freq_table)
# children_freq_table.to_csv('children_freq_table.csv', index=False)


'''9. Provide percent frequency of Number of Children for the following categories separately:
a. For customers with age in [20, 40]
b. For customers with age in (40, 80]'''
age_ranges = {
    '20-40': (20, 40),
    '40-80': (40, 80)
}

result_data = []  # List to store results

# Calculate percent frequency of Number of Children for each age range
for label, age_range in age_ranges.items():
    lower, upper = age_range
    filtered_data = customer_df[(customer_df['Age'] > lower) & (customer_df['Age'] <= upper)]
    
    total_customers = len(filtered_data)
    children_freq = filtered_data['Number of Children'].value_counts(normalize=True) * 100
    
    for children, percent in children_freq.items():
        result_data.append({'Age Range': label, 'Number of Children': children, 'Percent Frequency': percent})

children_freq_by_age = pd.DataFrame(result_data)
print("\n Children frequency by Age Table:")
display(children_freq_by_age)

# children_freq_by_age.to_csv('children_freq_by_age.csv', index=False)


'''10. Frequency table for Education Level of the customers (category, number of observations as 
frequency, percent frequency, cumulative frequency, cumulative percent frequency)'''
education_freq_table = customer_df['Education Level'].value_counts(dropna=False).reset_index().sort_values(by='Education Level')
education_freq_table.columns = ['Education Level', 'Frequency']
education_freq_table['Percent Frequency'] = (education_freq_table['Frequency'] / len(customer_df)) * 100
education_freq_table['Cumulative Frequency'] = education_freq_table['Frequency'].cumsum()
education_freq_table['Cumulative Percent Frequency'] = (education_freq_table['Cumulative Frequency'] / len(customer_df)) * 100

print("\n Education frequency Table:")
display(education_freq_table)

# education_freq_table.to_csv('education_freq_table.csv', index=False)


'''11. Provide percent frequency of Education Level for the following categories separately:
a. For customers with age in [20, 25]
b. For customers with age in (25, 35]
c. For customers with age in (35, 80]'''
age_ranges = {
    '20-25': (20, 25),
    '25-35': (25, 35),
    '35-80': (35, 80)
}

result_data = []  # List to store results

# Calculate percent frequency of Education Level for each age range
for label, age_range in age_ranges.items():
    lower, upper = age_range
    filtered_data = customer_df[(customer_df['Age'] >= lower) & (customer_df['Age'] <= upper)]
    
    total_customers = len(filtered_data)
    education_freq = filtered_data['Education Level'].value_counts(normalize=True) * 100
    
    for education, percent in education_freq.items():
        result_data.append({'Age Range': label, 'Education Level': education, 'Percent Frequency': percent})

education_freq_by_age = pd.DataFrame(result_data)
print("\n Education frequency by age Table:")
display(education_freq_by_age)
# education_freq_by_age.to_csv('education_freq_by_age.csv',index=False)


'''12. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Annual Income of 
customers.'''
income_stats = customer_df['Annual Income'].describe()
print("\n Income Stats:")
display(income_stats)
# income_stats.to_csv('income_stats.csv',index=False)


'''13. Frequency table for Number of Accounts of the customers (category, number of observations 
as frequency, percent frequency, cumulative frequency, cumulative percent frequency)'''
accounts_freq_table = customer_df['Number of Accounts'].value_counts().reset_index().sort_values(by='Number of Accounts')
accounts_freq_table.columns = ['Number of Accounts', 'Frequency']
accounts_freq_table['Percent Frequency'] = (accounts_freq_table['Frequency'] / len(customer_df)) * 100
accounts_freq_table['Cumulative Frequency'] = accounts_freq_table['Frequency'].cumsum()
accounts_freq_table['Cumulative Percent Frequency'] = (accounts_freq_table['Cumulative Frequency'] / len(customer_df)) * 100

print("\n Number of accounts frequency")
display(accounts_freq_table)
# accounts_freq_table.to_csv('accounts_freq_table.csv',index=False)


'''14. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Total Credit Line of 
customers.'''
credit_line_stats = customer_df['Total Credit Line'].describe()
print("\n Credit Line Stats: ")
display(credit_line_stats)
# credit_line_stats.to_csv('credit_line_stats.csv', index=False)


Number of customers:  20000

Minimum Customer id 1000401   Maximum Customer id 9999879

 Number of Unique customer Ids:  20000

 Stats on Customer Age:


count    20000.000000
mean        50.449000
std         17.253941
min         21.000000
25%         36.000000
50%         50.000000
75%         65.000000
max         80.000000
Name: Age, dtype: float64

,Gender,Frequency,Percent Frequency,Cumulative Frequency,Cumulative Percent Frequency
0,Female,10011,50.055,10011,50.055
1,Male,9989,49.945,20000,100.000



 Martial Status frequency Table:


,Martial Status,Frequency,Percent Frequency,Cumulative Frequency,Cumulative Percent Frequency
0,Married,11585,57.925,11585,57.925
1,Single,8415,42.075,20000,100.000


...

count     20000.000000
mean      83050.864000
std       21186.293785
min       39936.000000
25%       68224.000000
50%       80704.000000
75%       94848.000000
max      147680.000000
Name: Annual Income, dtype: float64


 Number of accounts frequency


,Number of Accounts,Frequency,Percent Frequency,Cumulative Frequency,Cumulative Percent Frequency
0,1,8415,42.075,8415,42.075
3,2,2151,10.755,10566,52.830
1,3,3435,17.175,14001,70.005
2,4,3128,15.640,17129,85.645
4,5,2011,10.055,19140,95.700
5,6,860,4.300,20000,100.000



 Credit Line Stats: 


count    20000.000000
mean     21280.746240
std      15060.839706
min       3993.600000
25%       8320.000000
50%      16723.200000
75%      30825.600000
max      88608.000000
Name: Total Credit Line, dtype: float64

**B. Functional Testing for Accounts information**

In [67]:
'''1. The minimum and maximum for the Date Opened across the entire cohort'''
min_date_opened = min(pd.to_datetime(customer_df['Accounts'].apply(lambda x: min([item['Date Opened'] for item in x]))))
max_date_opened = max(pd.to_datetime(customer_df['Accounts'].apply(lambda x: max([item['Date Opened'] for item in x]))))
print(f"\n Min date opened:{min_date_opened},   Max date opened:{max_date_opened}")


'''2. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Age of the accounts (as 
of January 1st, 2022, calculated in years)'''
current_year = 2021
account_ages = current_year - pd.to_datetime(customer_df['Accounts'].apply(lambda x: min([item['Date Opened'] for item in x])).str.strip(), format='%Y-%m-%d').dt.year
account_age_stats = account_ages.describe(percentiles=[.25, .5, .75])
print("\n Account Age stats:")
display(account_age_stats)
# account_age_stats.to_csv('account_age_stats.csv',index=True)


'''3. Frequency table for Account Age Flag (category, number of observations as frequency, 
percent frequency, cumulative frequency, cumulative percent frequency) (Account Age Flag 
is column in your data with values either True or False; the condition evaluated to get the 
Boolean is: (Customer’s Age – Account’s Age >= 20)'''
customer_df['Account Age Flag'] = (account_ages >= 20)
account_age_flag_freq_table = customer_df['Account Age Flag'].value_counts().reset_index()
account_age_flag_freq_table.columns = ['Account Age Flag', 'Frequency']
account_age_flag_freq_table['Percent Frequency'] = (account_age_flag_freq_table['Frequency'] / len(customer_df)) * 100
account_age_flag_freq_table['Cumulative Frequency'] = account_age_flag_freq_table['Frequency'].cumsum()
account_age_flag_freq_table['Cumulative Percent Frequency'] = (account_age_flag_freq_table['Cumulative Frequency'] / len(customer_df)) * 100


print("\n Account age flag frequency table:")
display(account_age_flag_freq_table)
# account_age_flag_freq.to_csv('account_age_flag_freq.csv',index=True)


'''4. The minimum and maximum Account Number in your results'''
min_account_number = min(customer_df['Accounts'].apply(lambda x: min([item['Account Number'] for item in x])))
max_account_number = max(customer_df['Accounts'].apply(lambda x: max([item['Account Number'] for item in x])))
print(f"\n Minimum Account Number :{min_account_number}, Maximum Account Number : {max_account_number}")


'''5. Frequency table for last digit of the Account Number (category, number of observations as 
frequency, percent frequency, cumulative frequency, cumulative percent frequency)'''
last_digit_freq = pd.Series([str(num[-1])[-1] for num in customer_df['Accounts'].apply(lambda x: [item['Account Number'] for item in x])])
last_digit_freq_table = last_digit_freq.value_counts().reset_index()
last_digit_freq_table.columns = ['Last Digit', 'Frequency']
last_digit_freq_table['Percent Frequency'] = (last_digit_freq_table['Frequency'] / len(customer_df)) * 100
last_digit_freq_table['Cumulative Frequency'] = last_digit_freq_table['Frequency'].cumsum()
last_digit_freq_table['Cumulative Percent Frequency'] = (last_digit_freq_table['Cumulative Frequency'] / len(customer_df)) * 100

print("\n Last digit frequency table:")
display(last_digit_freq_table)


'''6. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Account Credit Line.'''
account_credit_lines = customer_df['Accounts'].apply(lambda x: [item['Credit Line'] for item in x]).sum()
account_credit_line_stats = pd.Series(account_credit_lines).describe(percentiles=[.25, .5, .75])
print("\n Account credit Lines Stats:")
display(account_credit_line_stats)


'''7. Frequency table for Account Credit Line Flag (category, number of observations as 
frequency, percent frequency, cumulative frequency, cumulative percent frequency) 
(Account Credit Line Flag is column in your data with values either True or False; the 
condition evaluated to get the Boolean is: (sum(Account Credit Lines)==Total 
Credit Line)'''
customer_df['AccountCreditLineFlag'] = (sum(account_credit_lines) == customer_df['Total Credit Line'])
# Creating the frequency table for Account Credit Line Flag
account_credit_line_flag_freq = customer_df['AccountCreditLineFlag'].value_counts().reset_index()
account_credit_line_flag_freq.columns = ['Account Credit Line Flag', 'Frequency']
account_credit_line_flag_freq['Percent Frequency'] = (account_credit_line_flag_freq['Frequency'] / len(customer_df)) * 100
account_credit_line_flag_freq['Cumulative Frequency'] = account_credit_line_flag_freq['Frequency'].cumsum()
account_credit_line_flag_freq['Cumulative Percent Frequency'] = (account_credit_line_flag_freq['Cumulative Frequency'] / len(customer_df)) * 100

print("Account Credit LIne flag frequency table:")
display(account_credit_line_flag_freq)


'''8. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Annual Fee'''
annual_fees = customer_df['Accounts'].apply(lambda x: [item['Annual Fee'] for item in x]).sum()
annual_fee_stats = pd.Series(annual_fees).describe(percentiles=[.25, .5, .75])
print("\n Annual fee Stats:")
display(annual_fee_stats)
# annual_fee_stats.to_csv('annual_fee_stats.csv',index=True)


'''9. Frequency table for Annual Fee Flag (category, number of observations as frequency, percent 
frequency, cumulative frequency, cumulative percent frequency) (Account Fee Flag is 
column in your data with values either True or False; the condition evaluated to get the 
Boolean is: (Annual Fee == Account Credit Line × 0.01)'''
customer_df['Annual Fee Flag'] =(annual_fees == sum(account_credit_lines) * 0.01)


# Creating the frequency table for Annual Fee Flag
annual_fee_flag_freq = customer_df['Annual Fee Flag'].value_counts().reset_index()
annual_fee_flag_freq.columns = ['Annual Fee Flag', 'Frequency']
annual_fee_flag_freq['Percent Frequency'] = (annual_fee_flag_freq['Frequency'] / len(customer_df)) * 100
annual_fee_flag_freq['Cumulative Frequency'] = annual_fee_flag_freq['Frequency'].cumsum()
annual_fee_flag_freq['Cumulative Percent Frequency'] = (annual_fee_flag_freq['Cumulative Frequency'] / len(customer_df)) * 100

print("\n Annual fee flag Frequency:")
display(annual_fee_flag_freq)


'''10. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Annual Interest Rate'''
annual_interest_rates = [item['Annual Interest Rate']*100 for account in customer_df['Accounts'] for item in account ]
annual_interest_rate_stats = pd.Series(annual_interest_rates).describe()
print("\n Annual interest rate Stats:")
display(annual_interest_rate_stats)
# annual_interest_rate_stats.to_csv('annual_interest_rate_stats.csv',index=True)


 Min date opened:1942-01-01 00:00:00,   Max date opened:2001-12-31 00:00:00

 Account Age stats:


count    20000.000000
mean        49.449000
std         17.253941
min         20.000000
25%         35.000000
50%         49.000000
75%         64.000000
max         79.000000
Name: Accounts, dtype: float64


 Account age flag frequency table:


,Account Age Flag,Frequency,Percent Frequency,Cumulative Frequency,Cumulative Percent Frequency
0,True,20000,100.0,20000,100.0



 Minimum Account Number :10004011, Maximum Account Number : 99998791

 Last digit frequency table:


...

count    50749.000000
mean        82.448755
std         61.995523
min          2.062875
25%         32.740925
50%         67.347436
75%        117.710610
max        429.873234
dtype: float64


 Annual fee flag Frequency:


,Annual Fee Flag,Frequency,Percent Frequency,Cumulative Frequency,Cumulative Percent Frequency
0,False,20000,100.0,20000,100.0



 Annual interest rate Stats:


count    50749.000000
mean        22.509533
std          4.334031
min         15.000769
25%         18.770053
50%         22.523550
75%         26.247033
max         29.999331
dtype: float64

**C. Functional testing for Account Activity information**

In [68]:
#Stats calculation for account simulation
number_of_purchases=[]
number_of_cash_advances=[]
purchase_amounts=[]
cash_advance_amount=[]
payment_amount=[]
min_due_amount_info=[]
monthly_purchase_stats,monthly_cash_advance_stats,  monthly_payment_stats, monthly_interest_stats =[] ,[],[],[]
for customer in customers:
    for account in customer["Accounts"]:
        df_transactions=pd.DataFrame(account["Transactions"])
        number_of_purchases.append(len(df_transactions[df_transactions['Type'] == 'Purchase']))
        purchase_amounts.extend(df_transactions[df_transactions['Type'] == 'Purchase']["PurchaseAmount"])
        if 'CashAdvanceAmount' in df_transactions.columns:
            number_of_cash_advances.append(len(df_transactions[df_transactions['Type'] == 'Cash Advance']))
            cash_advance_amount.extend(df_transactions['CashAdvanceAmount'])
        if 'PaymentAmount' in df_transactions.columns:
            payment_amount.extend(df_transactions[df_transactions['Type'] == 'Payment']["PaymentAmount"])
        min_due_amount_info.append(account["MonthlyDetails"]["MinimumAmountDue"])
        monthly_purchase_stats.append(account["MonthlyDetails"]['TotalPurchasesMonth'])
        monthly_cash_advance_stats.append(account["MonthlyDetails"]['TotalCashAdvancesMonth'])
        monthly_payment_stats.append(account["MonthlyDetails"]['TotalPaymentsMonth'])
        monthly_interest_stats.append(account["MonthlyDetails"]['TotalInterestsChargedMonth'])

'''1. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Number of Transactions 
(Purchase or Cash Advance) for each card during the activity period.'''
num_of_transactions = pd.Series([ len(account["Transactions"]) for customer in customers for account in customer['Accounts'] ])
num_of_transactions_stats = num_of_transactions.describe()
print("\n Number of Transactions Stats:")
display(num_of_transactions_stats)


'''2. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Number of Purchases.'''
number_of_purchases_stats = pd.Series(number_of_purchases).describe()
print("\n Number of Purchases Stats:")
display(number_of_purchases_stats)


'''3. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Number of Cash 
Advances'''
number_of_cash_advances = pd.Series(number_of_cash_advances).describe()
print("\n Number of Cash Advances:")
display(number_of_cash_advances)


'''4. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Purchase Amounts.'''
purchase_amounts_stats = pd.Series(purchase_amounts).describe()
print("\n Purchase Amount Stats:")
display(purchase_amounts_stats)


'''5. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Cash Advance 
Amounts.'''
cash_advance_stats = pd.Series(cash_advance_amount).describe()
print("\n Cash Advance Stats:")
display(cash_advance_stats)


'''6. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Payments Amounts.'''
payment_amount_stats = pd.Series(payment_amount).describe()
print("\n Payment Amount Stats:")
display(payment_amount_stats)


'''7. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Closing Balances.'''
closing_balance_stats = pd.Series([account["CurrentBalance"] for customer in customers for account in customer["Accounts"]]).describe()
print("\n Closing Balance Stats:")
display(closing_balance_stats)


'''8. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Minimum Amounts Due'''
min_amount_due_stats = pd.Series(min_due_amount_info).describe()
print("\n Minimum Amounts Due stats:")
display(min_amount_due_stats)


'''9. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Total Purchase 
Amounts of the month.'''
monthly_purchase = pd.Series(monthly_purchase_stats).describe()
print("\n Monthly Purchases Stats:")
display(monthly_purchase)


'''10. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Total Cash Advance 
Amounts of the month.'''
monthly_cash_advance = pd.Series(monthly_cash_advance_stats).describe()
print("\n Monthly Cash Advances Stats:")
display(monthly_cash_advance)


'''11. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Payment Amounts 
of the month.'''
monthly_payment = pd.Series(monthly_payment_stats).describe()
print("\n Monthly Payments Stats:")
display(monthly_payment)


'''12. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Total Interests of the 
month.'''
monthly_interest = pd.Series(monthly_interest_stats).describe()
print("\n Monthly Interest Stats:")
display(monthly_interest)


'''13. Frequency table for Delinquency Counter (category, number of observations as frequency, 
# percent frequency, cumulative frequency, cumulative percent frequency)'''

delinquency_frequency = pd.Series(account["DelinquencyCounter"] for customer in customers for account in customer["Accounts"]).value_counts().reset_index()
delinquency_frequency.columns = ['Delinquency Counter', 'Frequency']
delinquency_frequency['Percent Frequency'] = (delinquency_frequency['Frequency'] / len(customer_df)) * 100
delinquency_frequency = delinquency_frequency.sort_values(by='Delinquency Counter')
delinquency_frequency['Cumulative Frequency'] = delinquency_frequency['Frequency'].cumsum()
delinquency_frequency['Cumulative Percent Frequency'] = (delinquency_frequency['Cumulative Frequency'] / 
                                                         delinquency_frequency['Frequency'].sum()) * 100

print("\n Delinquency frequency: ")
display(delinquency_frequency)


'''14. Frequency table for Annual Active Flag (category, number of observations as frequency, 
percent frequency, cumulative frequency, cumulative percent frequency) (Account Active 
Flag is column in your data with values either True or False; the condition evaluated to get 
the Boolean is: (Delinquency Counter < 3)'''
customer_df['Annual Active Flag'] = customer_df['Accounts'].apply(lambda x: any(account['DelinquencyCounter'] < 3 for account in x))

# Creating the frequency table for Annual Active Flag
annual_active_flag_freq_table = customer_df['Annual Active Flag'].value_counts().reset_index()
annual_active_flag_freq_table.columns = ['Annual Active Flag', 'Frequency']
annual_active_flag_freq_table['Percent Frequency'] = (annual_active_flag_freq_table['Frequency'] / len(customer_df)) * 100
annual_active_flag_freq_table['Cumulative Frequency'] = annual_active_flag_freq_table['Frequency'].cumsum()
annual_active_flag_freq_table['Cumulative Percent Frequency'] = (annual_active_flag_freq_table['Cumulative Frequency'] / len(customer_df)) * 100
display(annual_active_flag_freq_table)


 Number of Transactions Stats:


count    50749.000000
mean       150.272183
std         38.011888
min          8.000000
25%        139.000000
50%        163.000000
75%        175.000000
max        226.000000
dtype: float64


 Number of Purchases Stats:


count    50749.000000
mean        72.650101
std         18.797650
min          3.000000
25%         66.000000
50%         79.000000
75%         85.000000
max        112.000000
dtype: float64


 Number of Cash Advances:


...


 Monthly Interest Stats:


count    50749.000000
mean         7.875662
std         71.640826
min          0.000000
25%          0.000000
50%          0.000012
75%          8.774319
max       6967.956941
dtype: float64


 Delinquency frequency: 


,Delinquency Counter,Frequency,Percent Frequency,Cumulative Frequency,Cumulative Percent Frequency
3,0,10389,51.945,10389,20.471339
1,1,11749,58.745,22138,43.622534
2,2,10976,54.880,33114,65.250547
0,3,17635,88.175,50749,100.000000


,Annual Active Flag,Frequency,Percent Frequency,Cumulative Frequency,Cumulative Percent Frequency
0,True,16654,83.27,16654,83.27
1,False,3346,16.73,20000,100.00


**Storing all the above calculated stats and the generated customer data into one excel sheet**

In [69]:
#Saving all required tables in excel sheet
with pd.ExcelWriter('Output Data.xlsx', engine='openpyxl') as writer:
    #stats tables
        num_of_transactions_stats.to_excel(writer, sheet_name='Number of Transactions', index=True)
        age_stats.to_excel(writer, sheet_name='Age Stats', index=True)
        income_stats.to_excel(writer, sheet_name='Income Stats', index=True)
        credit_line_stats.to_excel(writer, sheet_name='credit_line_stats', index=True)
        account_age_stats.to_excel(writer, sheet_name='account_age_stats', index=True)
        annual_fee_stats.to_excel(writer, sheet_name='annual_fee_stats', index=True)
        annual_interest_rate_stats.to_excel(writer, sheet_name='annual_interest_rate_stats', index=True)
        num_of_transactions_stats.to_excel(writer, sheet_name='num_of_transactions_stats ', index=True)
        number_of_purchases_stats.to_excel(writer, sheet_name='number_of_purchases_stats ', index=True)
        number_of_cash_advances.to_excel(writer, sheet_name='number_of_cash_advances ', index=True)
        purchase_amounts_stats.to_excel(writer, sheet_name='purchase_amounts_stats ', index=True)
        cash_advance_stats.to_excel(writer, sheet_name='cash_advance_stats ', index=True)
        payment_amount_stats.to_excel(writer, sheet_name='payment_amount_stats ', index=True)
        closing_balance_stats.to_excel(writer, sheet_name='closing_balance_stats ', index=True)
        monthly_cash_advance.to_excel(writer, sheet_name='Monthly Cash Advance ', index=True)
        monthly_purchase.to_excel(writer, sheet_name='Monthly Purchase ', index=True)
        monthly_payment.to_excel(writer, sheet_name='Monthly Payment ', index=True)
        monthly_interest.to_excel(writer, sheet_name='Monthly Interest ', index=True)
        min_amount_due_stats.to_excel(writer, sheet_name='Min Due Amount ', index=True)
        #Frequency Tables
        gender_freq_table.to_excel(writer, sheet_name='gender_freq_table ', index=True)
        marital_status_freq_table.to_excel(writer, sheet_name='marital_status_freq_table ', index=True)
        marital_status_freq_by_age.to_excel(writer, sheet_name='marital_status_freq_by_age ', index=True)
        children_freq_table.to_excel(writer, sheet_name='children_freq_table ', index=True)
        children_freq_by_age.to_excel(writer, sheet_name='children_freq_by_age ', index=True)
        education_freq_table.to_excel(writer, sheet_name='education_freq_table ', index=True)
        education_freq_by_age.to_excel(writer, sheet_name='education_freq_by_age ', index=True)
        accounts_freq_table.to_excel(writer, sheet_name='accounts_freq_table ', index=True)
        account_age_flag_freq_table.to_excel(writer, sheet_name='account_age_flag_freq ', index=True)
        last_digit_freq.to_excel(writer, sheet_name='last_digit_freq ', index=True)
        account_credit_line_flag_freq.to_excel(writer, sheet_name='account_credit_line_flag_freq ', index=True)
        annual_fee_flag_freq.to_excel(writer, sheet_name='annual_fee_flag_freq ', index=True)
        delinquency_frequency.to_excel(writer, sheet_name='delinquency_frequency_table ', index=True)
        annual_active_flag_freq_table.to_excel(writer, sheet_name='active_flag_frequency_table ', index=True)


In [70]:
#Storing customers data in a excel sheet
pd.DataFrame(customers[:100]).to_excel("Sample Customers Data.xlsx",index = True)


In [71]:
# End time measurement
end_time = time.time()
# Calculate execution time
execution_time = end_time - start_time
# Log total time and memory usage
memory_usage = psutil.virtual_memory().used / (1024 ** 2)  # in MB
logging.info(f"Execution time of customer_generation: {execution_time} seconds")
logging.info(f"Memory usage of customer_generation: {memory_usage} MB")
# Close the logging file handler
logging.shutdown()